In [ ]:
import os
import numpy as np
from xml.dom.minidom import parse

classes = ['helmet','head','person']

def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = './labels/' +  img_jpg_file_name + '.txt'
    print(save_file_name)
    #file_path = open(save_file_name, "a+")
    with open(save_file_name ,'a+') as file_path:
        for box in img_box:

            cls_num = classes.index(box[0])

            new_box = convert_annot(size, box[1:])

            file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

        file_path.flush()
        file_path.close()

def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    #print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
   
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
  

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_txt_file(img_xml_file, [img_w, img_h], img_box)

files = os.listdir('./annotations')
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    print(file_xml[0])
    get_xml_data('./annotations/', file_xml[0])

In [16]:
from sklearn.model_selection import train_test_split
image_list = os.listdir('./origin/images')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=42)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=42)
print('total =',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))

from shutil import copyfile
from pathlib import Path

def copy_data(file_list, img_labels_root, imgs_source, mode):

    root_file = Path( './images/'+  mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exit")
        os.makedirs(root_file)

    root_file = Path('./labels/' + mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exit")
        os.makedirs(root_file)

    for file in file_list:               
        img_name = file.replace('.png', '')        
        img_src_file = imgs_source + '/' + img_name + '.png'        
        label_src_file = img_labels_root + '/' + img_name + '.txt'

        #print(img_sor_file)
        #print(label_sor_file)
        # im = Image.open(rf"{img_sor_file}")
        # im.show()

        # Copy image
        DICT_DIR = './images/'  + mode
        img_dict_file = DICT_DIR + '/' + img_name + '.png'

        copyfile(img_src_file, img_dict_file)

        # Copy label
        DICT_DIR = './labels/' + mode
        img_dict_file = DICT_DIR + '/' + img_name + '.txt'
        copyfile(label_src_file, img_dict_file)

copy_data(train_list, './origin/labels', './origin/images', "train")
copy_data(val_list,   './origin/labels', './origin/images', "val")
copy_data(test_list,  './origin/labels', './origin/images', "test")

total = 5000
train : 4000
val   : 500
test  : 500


In [19]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [45]:
import yaml

dict_file = {'train':'F:\\yolo\\images\\train' ,
            'val': 'F:\\yolo\\images\\val',
            'nc' : '3',
            'names' : ['helmet','head','person']}

with open('F:\\yolo\\yolov5\\data\\hard_head.yaml', 'w+', encoding='utf-8') as file:
    documents = yaml.dump(dict_file, file)

In [40]:
!wandb disabled

'wandb'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [46]:
!python train.py --img 416 --batch 32 --epochs 30 --data F:\yolo\yolov5\data\hard_head.yaml --cfg F:\yolo\yolov5\models\yolov5s.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=F:\yolo\yolov5\models\yolov5s.yaml, data=F:\yolo\yolov5\data\hard_head.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=30, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 
YOLOv5  v7.0-56-gc0ca1d2 Python-3.9.15 torch-1.13.1+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0

In [ ]:
%pip install -qr requirements.txt

In [48]:
%pip install -qr requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [50]:
import os
print(os.getcwd())

f:\yolo\yolov5


In [59]:
!python detect.py --source F:\\yolo\\images\\test\\hard_hat_workers3.png --project F:\\yolo\\yolov5\\runs\\detect\\test --name test --weights F:\\yolo\\yolov5\\runs\\train\\exp5\\weights\\best.pt --conf 0.25

detect: weights=['F:\\\\yolo\\\\yolov5\\\\runs\\\\train\\\\exp5\\\\weights\\\\best.pt'], source=F:\\yolo\\images\\test\\hard_hat_workers3.png, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=F:\\yolo\\yolov5\\runs\\detect\\test, name=test, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-56-gc0ca1d2 Python-3.9.15 torch-1.13.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 F:\yolo\images\test\hard_hat_workers3.png: 640x640 3 helmets, 219.3ms
Speed: 1.1ms pre-process, 219.3ms inference, 0.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to F:\yolo\yolov5\runs\detect\test\test2


In [55]:
!python detect.py --source F:\\yolo\\video_test\\test.mp4 --weights F:\\yolo\\yolov5\\runs\\train\\exp5\\weights\\best.pt --conf 0.25

detect: weights=['F:\\\\yolo\\\\yolov5\\\\runs\\\\train\\\\exp5\\\\weights\\\\best.pt'], source=F:\\yolo\\video_test\\test.mp4, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-56-gc0ca1d2 Python-3.9.15 torch-1.13.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/22415) F:\yolo\video_test\test.mp4: 384x640 3 helmets, 127.8ms
video 1/1 (2/22415) F:\yolo\video_test\test.mp4: 384x640 3 helmets, 135.4ms
video 1/1 (3/22415) F:\yolo\video_test\test.mp4: 384x640 4 helmets, 124.7ms
video 1/1 (4/22415) F:\yolo\video_test\test.mp4: 384x640 3 helmets, 127.7ms
video 1/1 (5/22415) 